In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install ultralytics

In [2]:
from ultralytics import YOLO
import json
import glob
import os

In [8]:
directory_name = "gdrive/MyDrive/Colab Notebooks/GESTALT/"

In [10]:
def detect_tags_from_jpgs_in_directory(directory_name):
    # Find all jpegs in the directory
    list_of_img_files = glob.glob(os.path.join(directory_name,"*.jpg"))

    # Instantiate model
    model = YOLO("yolov8m.pt")

    # Run images through model and get results
    results = model.predict(list_of_img_files, verbose=False)

    # Check that we got results for every image
    assert len(results) == len(list_of_img_files)

    # Create blank dictionary
    objects = {}

    # Loop over each image
    for idx in range(len(list_of_img_files)):
        filename = list_of_img_files[idx]
        result = results[idx]
        tags = []
        coords = []
        probs = []
        # Loop over each object detected in the image
        for box in result.boxes:
            class_id = result.names[box.cls[0].item()]
            cords = box.xyxy[0].tolist()
            cords = [round(x) for x in cords]
            conf = round(box.conf[0].item(), 2)

            tags.append(class_id)  # object type like person or bench
            coords.append(cords)  # coordinates in the image like [121, 632, 207, 732]
            probs.append(conf)  # confidence score like 0.81

        objects[filename] = {"objects":tags, "coordinates":coords, "probabilities":probs}

    return json.dumps(objects)

In [11]:
print(detect_tags_from_jpgs_in_directory(directory_name))

{"gdrive/MyDrive/Colab Notebooks/GESTALT/example.jpg": {"objects": ["person", "person", "person", "bench", "bench", "person", "bench", "person", "bench", "person", "bench"], "coordinates": [[208, 621, 249, 720], [121, 632, 207, 732], [165, 627, 210, 726], [329, 658, 463, 703], [575, 651, 659, 682], [388, 630, 425, 699], [672, 649, 734, 675], [77, 630, 136, 725], [74, 682, 246, 728], [585, 616, 603, 651], [532, 642, 578, 688]], "probabilities": [0.85, 0.81, 0.77, 0.76, 0.76, 0.68, 0.61, 0.61, 0.49, 0.49, 0.26]}, "gdrive/MyDrive/Colab Notebooks/GESTALT/example2.jpg": {"objects": ["person", "person", "person", "bench", "bench", "person", "bench", "person", "bench", "person", "bench"], "coordinates": [[208, 621, 249, 720], [121, 632, 207, 732], [165, 627, 210, 726], [329, 658, 463, 703], [575, 651, 659, 682], [388, 630, 425, 699], [672, 649, 734, 675], [77, 630, 136, 725], [74, 682, 246, 728], [585, 616, 603, 651], [532, 642, 578, 688]], "probabilities": [0.85, 0.81, 0.77, 0.76, 0.76, 0.68